# Topografisk prominens

In [1]:
# Kan finne info og data her: https://github.com/akirmse/mountains
import geopandas as gpd
import pandas as pd
iso = pd.read_csv('C:/data/alliso-sorted.txt', sep=',', header=None)
#peaks = pd.read_csv('c:/data/all-peaks-sorted-p100.txt', sep=',', header=None)
iso.columns = ['latitude','longitude','elevation','key saddle latitude','key saddle longitude','prominence']
# start
iso = iso[iso['latitude']>55] 
iso = iso[iso['latitude']<72]
iso = iso[iso['longitude']>3]
iso = iso[iso['longitude']<30]
iso = gpd.GeoDataFrame(iso, geometry=gpd.points_from_xy(iso.longitude, iso.latitude))
iso.crs = "EPSG:4326"
iso = iso.to_crs("EPSG:25833")
iso.sindex

ModuleNotFoundError: No module named 'geopandas'

In [2]:
#FyKo = gpd.read_file("c:/data/Fylke_kommune_flate_32633.shp", use_arrow=True)
FyKo = gpd.read_file("c:/data/Fylke_kommune_flate.gpkg", use_arrow=True)
FyKo = FyKo.to_crs("EPSG:25833")
FyKo.sindex
FyKo = FyKo[['kommunenummer','navn','navn_2','fylkesnummer','geometry']]
FyKo.columns = ['kommunenummer','kommunenavn','fylkesnavn','fylkesnummer','geometry']
#FyKo.head()
FyKo['kommunenummer'] = FyKo['kommunenummer'].astype(str)
FyKo['fylkesnummer'] = FyKo['fylkesnummer'].astype(str)
FyKo['kommunenummer'] = FyKo['kommunenummer'].str.replace('.0', '')
FyKo['fylkesnummer'] = FyKo['fylkesnummer'].str.replace('.0', '')

c:\Users\endofs\AppData\Local\ESRI\conda\envs\envFjell\Lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()


In [4]:
from osgeo import ogr

driver = ogr.GetDriverByName("FileGDB")
if driver:
    print("FileGDB driver is available.")
else:
    print("FileGDB driver is not available.")

FileGDB driver is not available.


In [ ]:
tmp = iso.sjoin(FyKo, how='left', predicate = 'intersects')
tmp = tmp[~tmp['index_right'].isna()]

tmp['elevation'] = tmp['elevation'].astype(int)*0.3048# feet to meters
output = 'C:/data/norske_topper.gpkg'
#tmp.to_file('C:/data/norske_topper.gdb', layer='topper', driver="FileGDB")
#tmp.to_file('C:/data/norske_topper.gpkg', layer='topper', driver='GPKG')

DataSourceError: Could not obtain driver: FileGDB (check that it was installed correctly into GDAL)

In [5]:
tmp = tmp[tmp['elevation']>500]
tmp2 = tmp[1:100]
tmp2 = tmp2.to_crs("EPSG:4326")
# Lag en leaflet
import folium

# Create a Folium map centered around the points
m = folium.Map(location=[58, 10], zoom_start=5)

# Add points to the map with popups showing the "Prominence" values
for idx, row in tmp2.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=f"Høyde (prominens, kommune):{row['elevation']} ({row['prominence']}, {row['kommunenavn']})"
    ).add_to(m)

# Display the map
#m.save("map.html")
m

# Publisere resultat

In [6]:
output = 'C:/data/norske_topper.gdb'

In [2]:
from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayerCollection

from IPython.display import display
import keyring
import settings
import os
import shutil
#import gl_p

#output_zip = gl_p.catalog_db_result + '/' + gl_p.name_db_data_result
#output_zip = output_zip.replace('.gdb', '.zip')
#output = output_zip.replace('.zip', '')
#output_zip = "C:/prosjekt/Analyse/bonn/bonn.gdb.zip"
#print(output)

In [7]:
#shutil.make_archive(output1.replace('.gdb', ''), 'zip', output1)
# Denne førsteversjonen gjør at når man åpner .-zipfilen så kommer man rett i mappens innhold. AGOL må ha det slik at åpning av zip-filen viser først mappen "fil.gdb" før man så går inn i den
# shutil.make_archive(output1, format = 'zip', output1)
# forsøker alternativ tilnærming
shutil.make_archive(output, format = 'zip', root_dir=os.path.dirname(output), base_dir=os.path.basename(output))

'C:\\data\\norske_topper.gdb.zip'

Laster opp filgeodatabasene til Arcgis Online.

In [8]:
print("Kobler til AGOL")
#pw = keyring.get_password("ArcGISOnline", "endofs_mdir")# Vi kan nå hente passordet ut igjen
#gis = GIS("https://miljodirektoratet.maps.arcgis.com/","endofs_mdir",pw, verify_cert=False)
gis = GIS(url=settings.agolUrl, username=settings.agolUser, password=settings.agolPwd, verify_cert=True)

print("Prepper filen")
item_properties = {
    'title': 'Fjelltopper i Norge',
    'tags': 'Fjelltopper, Norge',
    'type': 'File Geodatabase',
    'description': 'Opplastingen er et resultat av Python Notebook hos Miljødata. Dataene er hentet fra https://github.com/akirmse/mountains.'
    }

# Last opp zippet filgeodatabase til Feature Layer i AGOL
print("Laster opp filen")
fgdb_item = gis.content.add(item_properties, output+".zip")
# Publish the uploaded file geodatabase as a hosted feature layer
published_layer = fgdb_item.publish()

Kobler til AGOL
Prepper filen
Laster opp filen


Må så gjøre om disse 'hosted feature layers' til et web map.

In [ ]:
from arcgis.mapping import WebMap

# Create a new web map
web_map = WebMap()
# TODO: WEBMAP får ikke et bakgrunnskart, så dette må settes i AGOL
# Add the hosted feature layer to the web map
web_map.add_layer(published_layer)

# Define web map properties
web_map_properties = {
    "title": "Fjelltopper i Norge",
    "snippet": "Opplastingen er et resultat av Python Notebook hos Miljødata. Dataene er hentet fra https://github.com/akirmse/mountains. Publisert fra VS Code.",
    "tags": "Fjelltopper, Norge, friluftsliv"
}

# Set correct web map basemap, it incorrectly adds ' web map' to the title, making it fail to load properly in AGOL
web_map.basemap_title("Basis (25833)")

# Save the web map
web_map_item = web_map.save(web_map_properties)

# Share the web map
web_map_item.share(everyone=True)

C:\Users\endofs\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has be removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


{'notSharedWith': [], 'itemId': 'a38245da17e5441e8ad9915828231d17'}